In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import math
import random
import time

In [54]:
food = pd.read_csv(r'data_saved.csv', encoding='utf-8')
food.head(30)

def save_loc(s):
    return (' '.join(s.split()[1:])).split('(')[0].strip()

food['글자수'] = food['맛집명'].apply(len)
food['주소']=food['주소'].apply(save_loc)

# 글자수를 기준으로 데이터프레임 정렬
sorted_food = food.sort_values(by='글자수', ascending=False)

sorted_food.head(50)

['괴정역' '교대역' '구서역' '남산역' '남포역' '낫개역' '노포역' '다대포항역' '다대포해수욕장역' '당리역' '대티역'
 '동대신역' '동래역' '동매역' '두실역' '명륜역' '범내골역' '범어사역' '범일역' '부산대역' '부산역' '부산진역'
 '부전역' '사하역' '서대신역' '서면역' '시청역' '신장림역' '신평역' '양정역' '연산역' '온천장역' '자갈치역'
 '장림역' '장전역' '전포역' '중앙역' '초량역' '토성역' '하단역' '가야역' '감전역' '개금역' '경성대부경대역'
 '광안역' '구남역' '금련산역' '남천역' '냉정역' '대연역' '덕천역' '덕포역' '동백역' '동의대역' '모라역' '못골역'
 '국제금융센터부산은행역' '문헌역' '문현역' '민락역' '부암역' '사상역' '센텀시티역' '수영역' '수정역' '벡스코역'
 '율리역' '장산역' '주례역' '중동역' '지게골역' '해운대역' '화명역' '강서구청역' '거제역' '구포역' '남산정역'
 '만덕역' '망미역' '물만골역' '미남역' '배산' '사직역' '숙등역' '종합운동장역' '금사역' '낙민역' '동부산대학역'
 '명장역' '반여농산물시장역' '서동역' '수안역' '안평역' '영산대역' '충렬사역']


In [88]:
food['인근역명'].unique

,호선,인근역명,맛집명,주소,구분,대표메뉴,출구번호,글자수
0,1,괴정역,샤브향괴정점,사하구 사하로 197번길10,일식,샤브샤브,6,6
1,1,괴정역,한가정,사하구 낙동대로 256,한식,갈비,1,3
2,1,괴정역,도담식당,사하구 낙동대로233번길 16,한식,복어,2,4
3,1,괴정역,추담골,사하구 낙동대로233번길 34,한식,추어탕,2,3
4,1,괴정역,고흥녹동세발낙지,사하구 원양로 376,한식,낙지,6,8
...,...,...,...,...,...,...,...,...
833,4,충렬사역,새미골해물아구찜,동래구 안연로 58-1,한식,아구찜,1,8
834,4,충렬사역,청송식당,동래구 충렬대로359번길 44,일식,회,4,4
835,4,충렬사역,원조찹쌀아구찜,동래구 반송로218번길 18,한식,아구찜,4,7
836,4,충렬사역,로타리삼계탕,동래구 충렬대로 368,한식,삼계탕,1,6


In [3]:
def separate_korean_syllables(korean_word):
    cho_sung =[
        'ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ',
        'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ'
    ]
    jung_sung = [
        'ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ',
        'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ'
    ]
    jong_sung = [
        '', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ',
        'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ',
        'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ'
    ]

    cho_sung_dic = {
        'ㄱ':'g', 'ㄲ':'k', 'ㄴ':'n', 'ㄷ':'d', 'ㄸ':'t', 'ㄹ':'l', 'ㅁ':'m', 'ㅂ':'b', 'ㅃ':'p', 'ㅅ':'s', 'ㅆ':'ss',
        'ㅇ':'', 'ㅈ':'j', 'ㅉ':'zz', 'ㅊ':'ch', 'ㅋ':'k', 'ㅌ':'t', 'ㅍ':'p', 'ㅎ':'h'
    }

    jung_sung_dic = {
        'ㅏ':'a', 'ㅐ':'e', 'ㅑ':'ya', 'ㅒ':'ye', 'ㅓ':'eo', 'ㅔ':'e', 'ㅕ':'yeo', 'ㅖ':'ye', 'ㅗ':'o', 'ㅘ':'wa', 'ㅙ':'we',
        'ㅚ':'oe', 'ㅛ':'yo', 'ㅜ':'u', 'ㅝ':'uo', 'ㅞ':'we', 'ㅟ':'wi', 'ㅠ':'yu', 'ㅡ':'u', 'ㅢ':'ui', 'ㅣ':'i'
    }

    jong_sung_dic = {
        '':'', 'ㄱ':'k', 'ㄲ':'k', 'ㄳ':'k', 'ㄴ':'n', 'ㄵ':'n', 'ㄶ':'n', 'ㄷ':'t', 'ㄹ':'r', 'ㄺ':'k', 'ㄻ':'m',
        'ㄼ':'b', 'ㄽ':'t', 'ㄾ':'r', 'ㄿ':'p', 'ㅀ':'r', 'ㅁ':'m', 'ㅂ':'b', 'ㅄ':'b', 'ㅅ':'t', 'ㅆ':'t', 'ㅇ':'ng',
        'ㅈ':'t', 'ㅊ':'t', 'ㅋ':'k', 'ㅌ':'t', 'ㅍ':'p', 'ㅎ':'t'
    }

    result = []
    for char in korean_word:
        if '가' <= char <= '힣':
            char_code = ord(char) - ord('가')
            cho_index = char_code // (21 * 28)
            jung_index = (char_code % (21 * 28)) // 28
            jong_index = char_code % 28
            cho = cho_sung[cho_index]
            jung = jung_sung[jung_index]
            jong = jong_sung[jong_index]
            result.append(cho_sung_dic[cho]+jung_sung_dic[jung]+jong_sung_dic[jong])
        else:
            result.append(char)
    
    return ''.join(result)

# 함수 테스트
input_word = '안녕하세요'
separated_syllables = separate_korean_syllables(input_word)
print(separated_syllables)

annyeonghaseyo


In [5]:
print(sorted(food['대표메뉴'].unique()))

['간장게장', '갈비', '갈치요리', '감자탕', '게낙찜', '경양식', '고기', '곰탕', '곱창', '광어', '국밥', '국수', '굴전복', '김치 매실두루치기 김치찜', '김치볶음밥', '낙지', '냉면', '대구탕', '돈가스', '돌솔밥', '돌솥밥', '동태찌게', '동태탕동태찜', '매운탕', '메기탕', '목황정', '미역국', '밀면', '바비큐', '보신탕', '보쌈', '복국', '복어', '복요리', '부대찌게 전골', '분식', '뷔페', '비빔밥', '뽈찜', '삼겹살', '삼계탕', '샐러드바', '생삽겹살', '생선구이', '샤브샤브', '선지국', '설렁탕', '송이버섯전골 버섯밥', '수육전골', '수제만두', '수제어묵', '순대', '순두부', '숯불구이', '스테이크', '스파게티', '식단표 별 상이', '쌈밥', '아구찜', '양대창구이', '양식 전문', '양지탕', '오리', '완당', '우동', '우삼겹', '월남쌈', '유산슬 오향장육', '육개장', '장어', '재첩국', '전복구이', '정식', '조개전골', '족발', '중식', '짜장면', '짬뽕', '쭈꾸미두루치기', '쭈꾸미볶음', '찌개', '초밥', '추어탕', '치킨', '코다리찜', '코스요리', '타르트', '탕수육', '한식코스', '한우', '해물 생선요리', '해물찜', '해물탕', '해장국', '황태요리', '황태탕 황태구이', '회']


In [6]:
def save_loc(s):
    s=' '.join(s.split()[1:]).split('(')[0]
    return separate_korean_syllables(s)

In [32]:
def choose_multiple(station, m, num_rows=1):
    L = food[(food['인근역명'].str.contains(station)) & (food['대표메뉴'] == m)]
    n = L.shape[0]
    assert n > 0, "조건을 만족하는 데이터가 없습니다."

    L=L[["맛집명", '주소']]

    row_lists = []
    for index, row in L.iterrows():
        row_lists.append(row.tolist())
    print(row_lists)
    random.shuffle(row_lists)
    row_lists*=3

    return row_lists[:num_rows]

def menu(station):
    L=food[food['인근역명'].str.contains(station)]
    return L['대표메뉴'].unique()
    
menu("동래")
choose_multiple("동래", "갈비")

[['대궐집', '동래구 동래로136번길 26'], ['세연정', '동래구 충렬대로 149']]


[['세연정', '동래구 충렬대로 149']]

In [55]:


#메뉴들 업로드

'''
db.collection('user')
print(food['인근역명'].unique())

for i in food['인근역명'].unique():
    ref=db.collection("station").document()
    ref.set({'station':i , 'menu': list(menu(i))})


#관광지 업로드
def settrip(station, trip):
    for doc in matching_collection.stream():
        doc_dict = doc.to_dict()
        if station in doc_dict.get('station'):
            doc.reference.update({'trip': trip})

def settrip()
'''

['괴정역' '교대역' '구서역' '남산역' '남포역' '낫개역' '노포역' '다대포항역' '다대포해수욕장역' '당리역' '대티역'
 '동대신역' '동래역' '동매역' '두실역' '명륜역' '범내골역' '범어사역' '범일역' '부산대역' '부산역' '부산진역'
 '부전역' '사하역' '서대신역' '서면역' '시청역' '신장림역' '신평역' '양정역' '연산역' '온천장역' '자갈치역'
 '장림역' '장전역' '전포역' '중앙역' '초량역' '토성역' '하단역' '가야역' '감전역' '개금역' '경성대부경대역'
 '광안역' '구남역' '금련산역' '남천역' '냉정역' '대연역' '덕천역' '덕포역' '동백역' '동의대역' '모라역' '못골역'
 '국제금융센터부산은행역' '문헌역' '문현역' '민락역' '부암역' '사상역' '센텀시티역' '수영역' '수정역' '벡스코역'
 '율리역' '장산역' '주례역' '중동역' '지게골역' '해운대역' '화명역' '강서구청역' '거제역' '구포역' '남산정역'
 '만덕역' '망미역' '물만골역' '미남역' '배산' '사직역' '숙등역' '종합운동장역' '금사역' '낙민역' '동부산대학역'
 '명장역' '반여농산물시장역' '서동역' '수안역' '안평역' '영산대역' '충렬사역']


"\n#관광지 업로드\ndef settrip(station, trip):\n    for doc in matching_collection.stream():\n        doc_dict = doc.to_dict()\n        if station in doc_dict.get('station'):\n            doc.reference.update({'trip': trip})\n\ndef settrip()\n"

In [20]:
menu("동래")
for i in menu("동래"):
    print(i, end=' ')
    print(((food["대표메뉴"]==i) & (food["인근역명"].str.contains("동래"))).sum())

갈비 2
낙지 1
갈치요리 1
삼계탕 1
고기 1
초밥 1
감자탕 1
추어탕 2
짜장면 1


In [18]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account
cred = credentials.Certificate('key.json')
firebase_admin.initialize_app(cred)

db = firestore.client()

In [21]:
document = db.collection('user').document('김서윤2')
document.set({
    '역명': "동래",

})

'''
document = db.collection('user').document('김서윤2')
doc_data = document.get().to_dict()
'''



"\ndocument = db.collection('user').document('김서윤2')\ndoc_data = document.get().to_dict()\n"

In [26]:
user_collection = db.collection('user')
for doc in user_collection.stream():
    doc_id = doc.id
    print(f"문서 이름: {doc_id}")

    doc_data = doc.to_dict()
    print(doc_data)

문서 이름: Dluqd1riBkhVXzlJPpb1
{'station': '동래', 'uid': 'Dluqd1riBkhVXzlJPpb1', 'gender': 'male', 'meet': 'male', 'partner_uid': 'Dluqd1riBkhVXzlJPpb1', 'name': '김서윤2'}
문서 이름: PuQDEsmd6KjNQU0svmCS
{'station': '동래', 'uid': 'PuQDEsmd6KjNQU0svmCS', 'gender': 'male', 'meet': 'male', 'partner_uid': 'Dluqd1riBkhVXzlJPpb1', 'name': '김서윤'}
문서 이름: RrDEDz3gG0yfTUyASpeN
{'station': '동래', 'uid': ' RrDEDz3gG0yfTUyASpeN', 'meet': 'male', 'gender': 'male', 'partner_uid': 'RrDEDz3gG0yfTUyASpeN', 'name': '서윤4'}


In [59]:
#사람 궁합 체크

def uid_check(i, j):
    i_data=db.collection('user').document(i).get().to_dict()
    j_data=db.collection('user').document(j).get().to_dict()
    if (i_data['gender']==j_data['meetGender'] or j_data['meetGender']=="none") and (i_data['meetGender']==j_data['gender'] or i_data['meetGender']=="none"):
        return 1
    return 0


print(uid_check('eDNme6YntRpwvko5boFk', 'RrDEDz3gG0yfTUyASpeN'))

1


In [23]:
#uid -> 이름

def uid_to_name(uid):
    data=db.collection('user').document(uid).get().to_dict()
    return data['name']

uid_to_name("RrDEDz3gG0yfTUyASpeN")

'서윤4'

In [24]:
menu("동래") #역의 근처맛집

array(['갈비', '낙지', '갈치요리', '삼계탕', '고기', '초밥', '감자탕', '추어탕', '짜장면'],
      dtype=object)

In [103]:
#doc_ref = db.collection(collection_name).document(document_id)
#doc_ref.set(data)

#매칭되었을때

personal={
    "Chic":1,
    "Active":1,
    "Friendly":1,
    "Confident":1,
    "Lovely":1,
    "Quiet":-1,
    "Funny":1,
    "Attractive":1,
    "Independent":-1,
    "Curious":1,
    "Passionate":1,
    "Charismatic":1,
    "Kind":-1,
    "Calm":-1,
    "Relaxed":-1
}

def matched_add(i, j, MENU, station):
    path=db.collection('matched').document(i)
    db.collection('user').document(i).update({'partner_uid': i})
    db.collection('user').document(j).update({'partner_uid': i})
    score = sum([
    personal.get(p, 0) 
    for p in db.collection('user').document(i).get().to_dict()['description'] + 
             db.collection('user').document(j).get().to_dict()['description']
    ])
    value=choose_multiple(station, MENU, num_rows=1)[0]
    for doc in db.collection('station').stream():
        doc_id=doc.id
        doc_data = doc.to_dict()
        if station.strip() in doc_data['station']:
            if score>0:
                station_eng=doc_data['station_out']
                trip=doc_data['trip_out']
                trip_loc=doc_data['loc_out']
                explain=doc_data['explain_out']
            else:
                station_eng=doc_data['station_in']
                trip=doc_data['trip_in']
                trip_loc=doc_data['loc_in']
                explain=doc_data['explain_in']
    path.set({
        i:j,
        j:i,
        'menu':MENU,
        i+'_choice':0,
        j+'_choice':0,
        'eat': value[0],
        'loc': value[1],
        'info':[station_eng, trip, trip_loc, explain]
    })

matched_add("W2X0IRN6QBa4abF4NnjU", "eDNme6YntRpwvko5boFk", "짜장면", "동래")

[['동보성', '동래구 온천천로 193']]


In [38]:
#main 루프문

matching_collection = db.collection('matching')
def Server():
    dic={}
    for doc in matching_collection.stream():
        doc_id = doc.id
        #print(f"문서 이름: {doc_id}")

        doc_data = doc.to_dict()
        menu_list=doc_data['menu']
        people_list=doc_data['people']
        for i in range(min(len(menu_list), len(people_list))):
            if dic.get(menu_list[i]):
                print(people_list[i], dic[menu_list[i]])
                if uid_check(people_list[i], dic[menu_list[i]]):
                    matched_add(people_list[i], dic[menu_list[i]], menu_list[i], doc_data['station'])
                    people_list[people_list.index(dic[menu_list[i]])]=menu_list[menu_list.index(menu_list[i])]=''
                    del dic[menu_list[i]]
                    people_list[i]=menu_list[i]=''
            else:
                dic[menu_list[i]]=people_list[i]

        menu_list=[i for i in menu_list if i]
        people_list=[i for i in people_list if i]
        matching_collection.document(doc_id).update({'menu': menu_list, 'people':people_list})

In [102]:
while 1:
    try:
        Server()
    except:
        pass
    time.sleep(3)

eDNme6YntRpwvko5boFk RrDEDz3gG0yfTUyASpeN
eDNme6YntRpwvko5boFk RrDEDz3gG0yfTUyASpeN
eDNme6YntRpwvko5boFk RrDEDz3gG0yfTUyASpeN
eDNme6YntRpwvko5boFk RrDEDz3gG0yfTUyASpeN
eDNme6YntRpwvko5boFk RrDEDz3gG0yfTUyASpeN
eDNme6YntRpwvko5boFk RrDEDz3gG0yfTUyASpeN
eDNme6YntRpwvko5boFk RrDEDz3gG0yfTUyASpeN
eDNme6YntRpwvko5boFk RrDEDz3gG0yfTUyASpeN
eDNme6YntRpwvko5boFk RrDEDz3gG0yfTUyASpeN
eDNme6YntRpwvko5boFk RrDEDz3gG0yfTUyASpeN
eDNme6YntRpwvko5boFk RrDEDz3gG0yfTUyASpeN
eDNme6YntRpwvko5boFk  W2X0IRN6QBa4abF4NnjU
eDNme6YntRpwvko5boFk  W2X0IRN6QBa4abF4NnjU
eDNme6YntRpwvko5boFk  W2X0IRN6QBa4abF4NnjU
eDNme6YntRpwvko5boFk  W2X0IRN6QBa4abF4NnjU
 eDNme6YntRpwvko5boFk  W2X0IRN6QBa4abF4NnjU
 eDNme6YntRpwvko5boFk  W2X0IRN6QBa4abF4NnjU
eDNme6YntRpwvko5boFk  W2X0IRN6QBa4abF4NnjU
eDNme6YntRpwvko5boFk W2X0IRN6QBa4abF4NnjU
[['대궐집', '동래구 동래로136번길 26'], ['세연정', '동래구 충렬대로 149']]


KeyboardInterrupt: 

In [40]:
def insert(inputvalue):
    station, trip, loc, s, *_=inputvalue.strip().split('\n')
    print(station, loc, s,sep='\n')
    collection=db.collection('station')
    for doc in collection.stream():
        doc_id=doc.id
        doc_data = doc.to_dict()
        if station.strip() in doc_data['station']:
            print(doc_id)
            updated_data = {
                'station_eng': station,
                'explain': s,
                'location': loc,
                'trip': trip,
            }
            
            collection.document(doc_id).update(updated_data)
            print('succes!')
            break
    else:
        print("fail..")




In [63]:
trip_df

,역이름,장소이름,주소,설명
0,감전 (Gamjeon),감전 야생화 단지,"873 Gamjeon-dongSasang-gu, Busan",The Gamjeon Station Wildflower Preserve in Bus...
1,광안 (Gwangan),광안리 해수욕장,"219 Gwanganhaebyeon-roSuyeong-gu, Busan","Gwangan is home to Gwangalli Beach, which is c..."
2,NaN,장산,"1390 Jwa-dongHaeundae, Busan",Enjoy a three-hour outing to the top of Jangsa...
3,금련산 (GeumLeanMT),남천시장,"77 Gwanganhaebyeon-ro 15beon-gilSuyeong-gu, Busan",Gyeongnyeon Station Namcheon Market in Busan i...
4,NaN,경성대 문화골목,"36-1 Yongso-ro 13beon-gilNam-gu, Busan",Kyungsung University's Cultural Alley is a gre...
5,남천 (Namchun),남천 해변공원,"148-100 Namcheon-dong, Suyeong-gu, Busan",Nampcheon Station Beach Park in Busan is a ser...
6,남포 (Nampo),남포동,"Nampo-dong, Jung-gu, Busan","Nampo-dong, where you can spend all day and ne..."
7,NaN,흰여울 문화마을,"Huinnyeoul-gil, Jung-gu, Busan",Huinnyeoul Culture Village near Nampo Station ...
8,다대포항 (Dadaepo),다대포항,"375-17 Dadae-dong, Saha-gu, Busan",Dadaepo Station's Dadaepo Harbor in Busan is a...
9,다대포 해수욕장 (DadaepoBeach),감천 문화마을,"203 Gamnae 2(i)-ro, Saha-gu, Busan","Located near Dadaepo Beach Station in Busan, G..."


In [98]:
def station_to_rail(station):
    k=food[food['인근역명'].str.contains(station)]['호선'].unique()
    return int(sorted(k)[0])
station_to_rail('수안')

4

In [100]:
trip_df=pd.read_csv('output.csv', encoding='cp949')

trip_df.drop(['순서'], axis=1, inplace=True)

prev=''
collection=db.collection('station')
for index, row in trip_df.iterrows():
    if pd.isna(row['역이름']):
        val=prev
        print(val)
        for doc in collection.stream():
            doc_id=doc.id
            doc_data = doc.to_dict()
            if val in doc_data['station']:
                updated_data = {
                    'station_in': val,
                    'trip_in': row['장소이름'],
                    'loc_in': row['주소'],
                    'explain_in': row['설명']
                }
                collection.document(doc_id).update(updated_data)
                print('succes!')
                break
        else:
            print('fail')
    else:
        val=row['역이름'].split()[0]
        print(val)
        for doc in collection.stream():
            doc_id=doc.id
            doc_data = doc.to_dict()
            if val in doc_data['station']:
                updated_data = {
                    'station_out': row['역이름'].split()[1].strip()[1:-1],
                    'trip_out': row['장소이름'],
                    'loc_out': row['주소'],
                    'explain_out': row['설명'],
                    'station_in': row['역이름'].split()[1].strip()[1:-1],
                    'trip_in': row['장소이름'],
                    'loc_in': row['주소'],
                    'explain_in': row['설명'],
                    'rail': station_to_rail(val)
                }
                collection.document(doc_id).update(updated_data)
                print('succes!')
                break
        else:
            print('fail')
                
    prev=val
    assert type(prev)==str

감전
succes!
광안
succes!
광안
succes!
금련산
succes!
금련산
succes!
남천
succes!
남포
succes!
남포
succes!
다대포항
succes!
다대포
succes!
동대신
succes!
동래
succes!
문현
succes!
범어사
succes!
범일
succes!
부산
succes!
사직
succes!
서면
succes!
센텀시티
succes!
수영
succes!
온천장
succes!
자갈치
succes!
전포
succes!
중앙
succes!
중앙
succes!
해운대
succes!
초량
succes!
신평
succes!
충렬사
succes!


In [76]:
insert("""동래
동래읍성
부산광역시 동래구 명륜, 복천, 칠산, 명장, 안락동 일대
Dongnae Station's Dongnae Eupseong in Busan is a historic fortress that offers a glimpse into the city's past, with well-preserved walls and structures. This site provides insights into Busan's history and architecture.
""")

동래
부산광역시 동래구 명륜, 복천, 칠산, 명장, 안락동 일대
Dongnae Station's Dongnae Eupseong in Busan is a historic fortress that offers a glimpse into the city's past, with well-preserved walls and structures. This site provides insights into Busan's history and architecture.
B22JXxft4QdQWb8crutL
succes!


In [51]:

#초기화

def init():
    collection=db.collection('station')
    for doc in collection.stream():
        doc_id=doc.id
        doc_data = doc.to_dict()
        updated_data = {
            'station_eng': None,
            'explain': None,
            'location': None,
            'trip': None,
        }
        
        collection.document(doc_id).update(updated_data)
        print('succes!')
init()



succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!
succes!


In [92]:
food['인근역명'].unique()

array(['괴정역', '교대역', '구서역', '남산역', '남포역', '낫개역', '노포역', '다대포항역',
       '다대포해수욕장역', '당리역', '대티역', '동대신역', '동래역', '동매역', '두실역', '명륜역',
       '범내골역', '범어사역', '범일역', '부산대역', '부산역', '부산진역', '부전역', '사하역', '서대신역',
       '서면역', '시청역', '신장림역', '신평역', '양정역', '연산역', '온천장역', '자갈치역', '장림역',
       '장전역', '전포역', '중앙역', '초량역', '토성역', '하단역', '가야역', '감전역', '개금역',
       '경성대부경대역', '광안역', '구남역', '금련산역', '남천역', '냉정역', '대연역', '덕천역', '덕포역',
       '동백역', '동의대역', '모라역', '못골역', '국제금융센터부산은행역', '문헌역', '문현역', '민락역',
       '부암역', '사상역', '센텀시티역', '수영역', '수정역', '벡스코역', '율리역', '장산역', '주례역',
       '중동역', '지게골역', '해운대역', '화명역', '강서구청역', '거제역', '구포역', '남산정역', '만덕역',
       '망미역', '물만골역', '미남역', '배산', '사직역', '숙등역', '종합운동장역', '금사역', '낙민역',
       '동부산대학역', '명장역', '반여농산물시장역', '서동역', '수안역', '안평역', '영산대역', '충렬사역'],
      dtype=object)